In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'
VOCAB_SIZE = 2500


# Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ',dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ',dtype=int)

In [5]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258332
Nr of rows in test file 117774


In [6]:
print('Nr of emails in training file', np.unique(sparse_train_data[:,0]).size)

Nr of emails in training file 4013


In [7]:
print('Nr of emails in test file', np.unique(sparse_test_data[:,0]).size)

Nr of emails in test file 1724


In [8]:
sparse_train_data[-5:]

array([[5795, 2032,    0,    2],
       [5795, 2037,    0,    1],
       [5795, 2271,    0,    1],
       [5795, 2348,    0,    1],
       [5795, 2379,    0,    1]])

## How to create an Empty DataFrame

In [9]:
column_names =['DOC_ID']+ ['CATEGORY']+ list(range(0, VOCAB_SIZE) )
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index= index_names, columns= column_names)
full_train_data.fillna(0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0,word_idx =1 , cat_idx=2, freq_idx=3):
    """
    From a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments: 
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """

    column_names =['DOC_ID']+ ['CATEGORY']+ list(range(0, VOCAB_SIZE) )
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index= doc_id_names, columns= column_names)
    full_matrix.fillna(0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence

    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [15]:

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

In [16]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [17]:
prob_spam = full_train_data.CATEGORY.sum()/ full_train_data.shape[0]
print('Probability of spam is', prob_spam)

Probability of spam is 0.310989284824321


## Total Number of Words /Tokens

In [18]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [20]:
total_wc= email_lengths.sum()

## Number of Tokens in Spam & Ham Emails

In [21]:
spam_lengths= email_lengths[full_train_data.CATEGORY==1]
spam_wc= spam_lengths.sum()
ham_lengths= email_lengths[full_train_data.CATEGORY==0]
nonspam_wc= ham_lengths.sum()

In [22]:
email_lengths.shape[0]- spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [23]:
# which category of emails tend to be longer ? 
print('Average nr of words in spam emails {:.0f}'.format(spam_wc /spam_lengths.shape[0]))
print('Average nr of words in nonspam emails {:.3f}'.format(nonspam_wc /ham_lengths.shape[0]))

Average nr of words in spam emails 141
Average nr of words in nonspam emails 91.452


In [24]:
full_train_features.shape

(4013, 2500)

## Summing the Tokens Occuring in Spam & Ham

In [25]:
train_spam_tokens= full_train_features.loc[full_train_data.CATEGORY==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# laplace smoothing
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1
summed_spam_tokens.shape

(2500,)

In [28]:
summed_spam_tokens.tail()

2495     8
2496     1
2497     2
2498     1
2499    26
dtype: int64

In [46]:
train_ham_tokens= full_train_features.loc[full_train_data.CATEGORY==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1

# P(Token|Spam) - Probability that a Token Occurs githen the Email is Spam

In [47]:
prob_tokens_spam = summed_spam_tokens / (spam_wc+ VOCAB_SIZE)

# P(Token|Ham) - Probability that a Token Occurs githen the Email is Nonspam

In [49]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc+ VOCAB_SIZE)

# P(Token)- Probability that Token Occurs

In [50]:
prob_tokens_all= full_train_features.sum(axis=0) / total_wc

In [51]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [54]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [56]:
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

In [57]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [58]:
np.savetxt(TEST_FEATURE_MATRIX, x_test)
np.savetxt(TEST_TARGET_FILE, y_test)